# Master Feature Engineering Pipeline

Bu notebook, hiç bir alt notebook içeriğini değiştirmeden sırasıyla feature engineering adımlarını çalıştırır ve ana CSV dosyasına eklemeler yaparak kaydeder.

**Çalışma dizini**: Tüm alt `.ipynb` dosyalarının (`calculate_direction.ipynb`, `bounding_box.ipynb`, vb.) bulunduğu klasörde olmalıdır.

In [2]:
# Parameters
mode = "fake"
input_path = ""
output_path = ""


In [4]:
#!/usr/bin/env python3
import papermill as pm
from pathlib import Path
import logging

def main():
    logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s")
    log = logging.getLogger()

    # ─── 0) PROJECT_ROOT ayarı; gerektiğinde parents[i]’ı değiştir ────────
    PROJECT_ROOT = Path(__file__).resolve().parents[2]  
    log.info(f"Project root: {PROJECT_ROOT}")

    # ─── 1) Feature Notebook’lar ─────────────────────────────────────────
    NOTEBOOKS = [
        "calculate_direction.ipynb",
        "bounding_box.ipynb",
        "compute_avg_speed_math.ipynb",
        "compute_avg_acceleration.ipynb",
        "compute_idle_entropy_fixed2.ipynb",
        "compute_linearity_manual.ipynb",
    ]

    # ─── 2) Fake Data Ayarları ────────────────────────────────────────────
    FAKE_DATES = ["2025-04-21","2025-04-22","2025-04-23","2025-04-24","2025-04-25"]
    PARQUET_ROOT = PROJECT_ROOT / "parquet_dataset"
    ProcessFakeDir = PROJECT_ROOT / "processed" / "fake"
    ProcessFakeDir.mkdir(parents=True, exist_ok=True)

    # ─── 3) Real Data Ayarları ────────────────────────────────────────────
    REAL_DIR = PROJECT_ROOT / "data" / "real"
    ProcessRealAcerDir  = PROJECT_ROOT / "processed" / "real" / "acer"
    ProcessRealNatureDir = PROJECT_ROOT / "processed" / "real" / "nature"
    for d in (ProcessRealAcerDir, ProcessRealNatureDir):
        d.mkdir(parents=True, exist_ok=True)

    # ─── 4) Runs output ───────────────────────────────────────────────────
    RUNS_DIR = PROJECT_ROOT / "runs"
    RUNS_DIR.mkdir(exist_ok=True)

    # ─── 5) Fake için çalıştır ────────────────────────────────────────────
    for date in FAKE_DATES:
        input_pattern = str(PARQUET_ROOT / f"date={date}" / "*.parquet")
        output_csv    = ProcessFakeDir / f"{date}-processed.csv"

        for nb in NOTEBOOKS:
            out_nb = RUNS_DIR / f"fake_{date}_{nb}"
            log.info(f"→ Fake {date}: running {nb}")
            pm.execute_notebook(
                input_path = PROJECT_ROOT / nb,
                output_path= out_nb,
                parameters = {
                    "mode": "fake",
                    "input_path": input_pattern,
                    "output_path": str(output_csv)
                }
            )
        log.info(f"✅ Fake {date} done; output → {output_csv}")

    # ─── 6) Real için çalıştır ────────────────────────────────────────────
    for csv_path in sorted(REAL_DIR.glob("km_stat_*.csv")):
        stem   = csv_path.stem           # e.g. "km_stat_acer_20250421"
        parts  = stem.split("_")         # ["km","stat","acer","20250421"]
        prefix = parts[2]                # "acer" veya "nature"
        date   = parts[3]                # "20250421"
        output_csv = (
            ProcessRealAcerDir if prefix=="acer" else ProcessRealNatureDir
        ) / f"{date}-processed.csv"

        for nb in NOTEBOOKS:
            out_nb = RUNS_DIR / f"real_{prefix}_{date}_{nb}"
            log.info(f"→ Real {prefix} {date}: running {nb}")
            pm.execute_notebook(
                input_path = PROJECT_ROOT / nb,
                output_path= out_nb,
                parameters = {
                    "mode": "real",
                    "input_path": str(csv_path),
                    "output_path": str(output_csv)
                }
            )
        log.info(f"✅ Real {prefix} {date} done; output → {output_csv}")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'papermill'

In [ ]:
# 1) Directional Changes
%run calculate_direction.ipynb

In [ ]:
# 2) Bounding Box
%run bounding_box.ipynb

In [ ]:
# 3) Average Speed
%run compute_avg_speed_math.ipynb

In [ ]:
# 4) Average Acceleration
%run compute_avg_acceleration.ipynb

In [ ]:
# 5) Mouse Idle Ratio & Entropy
%run compute_idle_entropy_fixed2.ipynb

In [ ]:
# 6) Linearity
%run compute_linearity_manual.ipynb

In [ ]:
# Tamamlandı! 
print("Tüm feature engineering adımları tamamlandı. Ana CSV dosyalarına eklenen sütunları kontrol edin.")